In [2]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
#now doucuments / data_text has headline text and index

In [72]:
# print(len(documents))
print(documents[:10])
print(type(documents))

                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4
5                  ambitious olsson wins triple jump      5
6         antic delighted with record breaking barca      6
7  aussie qualifier stosur wastes four memphis match      7
8       aust addresses un security council over iraq      8
9         australia is locked into war timetable opp      9
<class 'pandas.core.frame.DataFrame'>


In [9]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [73]:
doc_sample = documents[documents['index'] == 0].values[0][0]
print(doc_sample)
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print(preprocess(doc_sample))
print(type(doc_sample))

aba decides against community broadcasting licence
['aba', 'decides', 'against', 'community', 'broadcasting', 'licence']
['decid', 'communiti', 'broadcast', 'licenc']
<class 'str'>


In [37]:
processed_docs = documents['headline_text'].map(preprocess)


In [74]:
print(len(processed_docs))
print(type(processed_docs))
print(processed_docs[:10])

1103663
<class 'pandas.core.series.Series'>
0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object


In [46]:
dictionary = gensim.corpora.Dictionary(processed_docs)


0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [54]:
print(dictionary)
# count = 0
# for k, v in dictionary.iteritems():
#     print(k, v)
#     count += 1
#     if count > 10:
#         break

Dictionary(14143 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [55]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print(dictionary)

Dictionary(14143 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [56]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [65]:
print(processed_docs[:3])
print (dictionary)
print(bow_corpus[0])

0     [decid, communiti, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
Name: headline_text, dtype: object
Dictionary(14143 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
[(0, 1), (1, 1), (2, 1), (3, 1)]


In [66]:
bow_doc_4310 = bow_corpus[0]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
                                                bow_doc_4310[i][1]))

Word 0 ("broadcast") appears 1 time.
Word 1 ("communiti") appears 1 time.
Word 2 ("decid") appears 1 time.
Word 3 ("licenc") appears 1 time.


In [67]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [68]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"high" + 0.015*"share" + 0.015*"school" + 0.015*"minist" + 0.014*"break" + 0.013*"student"
Topic: 1 
Words: 0.040*"australian" + 0.025*"adelaid" + 0.020*"test" + 0.017*"tasmanian" + 0.017*"open" + 0.017*"win" + 0.014*"melbourn" + 0.011*"vote" + 0.011*"darwin" + 0.009*"river"
Topic: 2 
Words: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.012*"show" + 0.012*"price" + 0.011*"search"
Topic: 3 
Words: 0.030*"govern" + 0.022*"south" + 0.020*"north" + 0.017*"countri" + 0.016*"fund" + 0.016*"rural" + 0.014*"hour" + 0.014*"west" + 0.013*"indigen" + 0.013*"australia"
Topic: 4 
Words: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold" + 0.014*"lose" + 0.013*"return" + 0.013*"park" + 0.012*"beat" + 0.011*"royal"
Topic: 5 
Words: 0.027*"elect" + 0.021*"sydney" + 0.017*"say" + 0.014*"labor" + 0.013*"time" + 0.013*"reco